In [ ]:
import os, shutil

os.environ["PATH"] = os.path.expanduser("~/.local/bin") + ":" + os.environ["PATH"]

if shutil.which("sciunit") is None:
    print("Sciunit is NOT installed. Please run the next cell.")
else:
    print("Sciunit is already installed and available.")


Only run the below cells if sciunit is not installed already

In [ ]:
import os, shutil, subprocess

os.environ["PATH"] = os.path.expanduser("~/.local/bin") + ":" + os.environ["PATH"]

if shutil.which("sciunit") is None:
    print("Installing Sciunit...")
    subprocess.run(["pip", "install", "--user", "sciunit2"], check=True)
else:
    print("Sciunit already installed.")

subprocess.run(["sciunit", "--version"], check=True)
